<a href="https://colab.research.google.com/github/Arif-Kasim1/PIAIC-201/blob/main/201_PROJECT_02_EXPERIMENT_A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -Uq langchain langchain-google-genai faiss-cpu google-generativeai typing-extensions langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 46.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 67.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.5 MB/s eta 0:00:00


In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains import RetrievalQA
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
import os
import textwrap
from google.colab import userdata

# Set your Google API key
os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY") # Set the environment variable to the correct API key

# Initialize the embeddings and LLM
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001",
                          google_api_key=userdata.get("GOOGLE_API_KEY"))
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0.3)

def read_articles(file_path):
    with open(file_path, 'r') as file:
        return file.read()

def create_rag_system():
    # Read and process the sample data
    raw_text = read_articles('/content/Data.txt')
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=50,
        separators=["\nTitle:", "\n\n", "\n", " ", ""]
    )
    texts = text_splitter.split_text(raw_text)

    # Create vector store
    # FAISS (Facebook AI Similarity Search) is a library for efficient
    # similarity search of dense vectors. Why FAISS? Fast similarity search
    # Efficient memory usage, Good for large datasets, Optimized for vector operations
    vector_store = FAISS.from_texts(texts, embeddings)

    # Create the RAG chain with chain type="stuff"
    # Stuff: Fastest but limited by context window
    # Map Reduce: Good for parallel processing of many documents
    # Refine: Most detailed but can be slower
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=vector_store.as_retriever(search_kwargs={"k": 3}),
        return_source_documents=True
    )

    return qa_chain

def create_general_chain():
    # Create a general purpose chain for non-RAG queries
    template = """Question: {question}

    Please provide a detailed answer."""

    prompt = PromptTemplate(
        input_variables=["question"],
        template=template,
    )

    general_chain = LLMChain(llm=llm, prompt=prompt)
    return general_chain

def ask_question(qa_chain, general_chain, question, use_rag=True):
    print("\nQuestion:", question)

    if use_rag:
        # Use RAG for document-specific questions
        response = qa_chain(question)
        print("\nAnswer (Using RAG):", textwrap.fill(response['result'], width=80))
        print("\nRelevant source chunks:")
        for i, doc in enumerate(response['source_documents'], 1):
            print(f"\nChunk {i}:")
            print(textwrap.fill(doc.page_content, width=80))
    else:
        # Use general LLM chain for non-document questions
        response = general_chain.run(question)
        print("\nAnswer (Using General LLM):", textwrap.fill(response, width=80))

def main():
    # Initialize both RAG and general systems
    rag_chain = create_rag_system()
    general_chain = create_general_chain()

    # Example questions - mix of RAG and non-RAG queries
    questions = [
        # RAG questions (about our documents)
        {"text": "What are the main advantages of cloud computing?", "use_rag": True},
        {"text": "How do neural networks learn to recognize images?", "use_rag": True},

        # Non-RAG questions (general knowledge)
        {"text": "Give me the coordinates of Eiffel Tower?", "use_rag": False},
        {"text": "A south asian guy is asking about rooh afza, what is that? in 50 words explanation", "use_rag": False}
    ]

    # Ask each question
    for q in questions:
        ask_question(rag_chain, general_chain, q["text"], q["use_rag"])
        print("\n" + "="*80 + "\n")


# The line if __name__ == "__main__": is a common Python idiom that determines
# whether a Python script is being run directly or being imported as a module
# into another script.
# Here's what it means:
# When you run a Python file directly (like python your_script.py):
# Python sets the special variable __name__ to "__main__"
# The code inside this if block will execute
# When you import the file as a module in another script:
# __name__ will be set to the module's name
# The code inside this if block won't execute

if __name__ == "__main__":
    main()

<ipython-input-2-3304bfdbd1dd>:64: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  general_chain = LLMChain(llm=llm, prompt=prompt)
<ipython-input-2-3304bfdbd1dd>:72: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = qa_chain(question)



Question: What are the main advantages of cloud computing?

Answer (Using RAG): Based on the provided text, the main advantages of cloud computing are
scalability, cost-effectiveness, and reduced maintenance overhead.

Relevant source chunks:

Chunk 1:
Article: Cloud computing has transformed how businesses operate in the digital
age. Instead of maintaining physical servers, companies can now rent computing
resources on-demand. This model offers several advantages: scalability, cost-
effectiveness, and reduced maintenance overhead. The three main service models
are Infrastructure as a Service (IaaS), Platform as a Service (PaaS), and
Software as a Service (SaaS). Major providers like AWS, Microsoft Azure, and
Google Cloud Platform compete to

Chunk 2:
Azure, and Google Cloud Platform compete to offer increasingly sophisticated
services, from basic storage to advanced AI capabilities.

Chunk 3:
Title: The Evolution of Cloud Computing



Question: How do neural networks learn to recogni

<ipython-input-2-3304bfdbd1dd>:80: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = general_chain.run(question)



Answer (Using General LLM): The Eiffel Tower doesn't have a single coordinate, as it's a structure with a
footprint and height.  To give coordinates, we need to specify a point.  The
most common and logical point to use is the **center of the base**.  The
coordinates of the center of the base of the Eiffel Tower are approximately:  *
**48.8584° N, 2.2945° E**  These coordinates are given in latitude and longitude
using the World Geodetic System 1984 (WGS 84) datum, which is the most commonly
used standard.  You can use these coordinates with any mapping software or GPS
device to locate the Eiffel Tower.  It's important to note that:  * **Slight
variations exist:**  Different mapping services might show slightly different
coordinates due to variations in surveying and data processing.  The difference
will be minimal, usually within a few meters. * **This is the base:** This
coordinate points to the ground level, at the center of the tower's base.  The
coordinates will be different if y